## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Currrent Description
0,0,Ushuaia,AR,-54.80,-68.30,59.00,54,75,18.34,2020-11-01 22:45:34,broken clouds
1,1,Tabriz,IR,38.08,46.29,51.80,29,0,6.93,2020-11-01 22:40:41,clear sky
2,2,Chokurdakh,RU,70.63,147.92,0.21,94,100,10.96,2020-11-01 22:45:34,light snow
3,3,Beringovskiy,RU,63.05,179.32,24.13,95,100,2.98,2020-11-01 22:45:35,overcast clouds
4,4,Faanui,PF,-16.48,-151.75,80.26,79,27,20.31,2020-11-01 22:45:35,light rain


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Currrent Description
4,4,Faanui,PF,-16.48,-151.75,80.26,79,27,20.31,2020-11-01 22:45:35,light rain
7,7,Basco,PH,20.45,121.97,79.90,81,100,29.71,2020-11-01 22:45:36,overcast clouds
9,9,Bengkulu,ID,-3.80,102.27,75.45,88,99,4.56,2020-11-01 22:45:36,overcast clouds
10,10,The Valley,AI,18.22,-63.06,82.99,74,40,16.11,2020-11-01 22:45:36,scattered clouds
11,11,Olinda,BR,-8.01,-34.86,80.60,74,40,5.82,2020-11-01 22:45:36,scattered clouds
15,15,Vaini,TO,-21.20,-175.20,78.80,78,75,12.75,2020-11-01 22:45:37,broken clouds
17,17,Daru,PG,-9.08,143.21,84.61,74,92,13.58,2020-11-01 22:45:38,overcast clouds
20,20,Sao Filipe,CV,14.90,-24.50,79.21,76,75,11.48,2020-11-01 22:45:39,broken clouds
25,25,Hilo,US,19.73,-155.09,86.00,58,40,6.93,2020-11-01 22:42:05,scattered clouds
32,32,Kavieng,PG,-2.57,150.80,85.69,67,1,5.75,2020-11-01 22:45:42,clear sky


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 4 to 576
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City_ID               179 non-null    int64  
 1   City                  179 non-null    object 
 2   Country               179 non-null    object 
 3   Lat                   179 non-null    float64
 4   Lng                   179 non-null    float64
 5   Max Temp              179 non-null    float64
 6   Humidity              179 non-null    int64  
 7   Cloudiness            179 non-null    int64  
 8   Wind Speed            179 non-null    float64
 9   Date                  179 non-null    object 
 10  Currrent Description  179 non-null    object 
dtypes: float64(4), int64(3), object(4)
memory usage: 16.8+ KB


In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Currrent Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Currrent Description,Lat,Lng,Hotel Name
4,Faanui,PF,80.26,light rain,-16.48,-151.75,
7,Basco,PH,79.90,overcast clouds,20.45,121.97,
9,Bengkulu,ID,75.45,overcast clouds,-3.80,102.27,
10,The Valley,AI,82.99,scattered clouds,18.22,-63.06,
11,Olinda,BR,80.60,scattered clouds,-8.01,-34.86,
15,Vaini,TO,78.80,broken clouds,-21.20,-175.20,
17,Daru,PG,84.61,overcast clouds,-9.08,143.21,
20,Sao Filipe,CV,79.21,broken clouds,14.90,-24.50,
25,Hilo,US,86.00,scattered clouds,19.73,-155.09,
32,Kavieng,PG,85.69,clear sky,-2.57,150.80,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna().drop_duplicates()
clean_hotel_df

,City,Country,Max Temp,Currrent Description,Lat,Lng,Hotel Name
4,Faanui,PF,80.26,light rain,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
7,Basco,PH,79.90,overcast clouds,20.45,121.97,Nathaniel's Lodge
9,Bengkulu,ID,75.45,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
10,The Valley,AI,82.99,scattered clouds,18.22,-63.06,CeBlue Villas & Beach Resort
11,Olinda,BR,80.60,scattered clouds,-8.01,-34.86,Hotel 5 Sóis
...,...,...,...,...,...,...,...
563,Comodoro Rivadavia,AR,77.00,clear sky,-45.87,-67.50,Austral Hotel
564,Trincomalee,LK,78.39,few clouds,8.57,81.23,Welcombe Hotel
569,Loandjili,CG,78.80,broken clouds,-4.76,11.86,Résidence ARCHANGE
570,Porto Novo,BJ,79.23,few clouds,6.50,2.60,Tour Eiffel Hotel Benin


In [29]:
# 8a. Create the output File (CSV)
output_data_file="Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Currrent Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))